In [1]:
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
import numpy
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

Using TensorFlow backend.


In [2]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset

dataframe = pandas.read_csv("normalAndSatan.csv")#, header=True) 

# samples 10000 random data points from 500k
dataframe = dataframe.sample(n=15000)
# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
dataset = dataframe_encoded.values

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
#print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors
print(num_of_classes)


2


In [5]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 10
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #optimizer=adam
    return model

In [6]:
#estimator = KerasClassifier(build_fn=baseline_model, epochs=32, batch_size=200, verbose=2)
estimator = baseline_model()
estimator.fit(X,dummy_y, epochs=100, batch_size=200,  verbose=2)
#kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
#y_pred = cross_val_predict(estimator, X, dummy_y, cv=kfold)
#results = cross_val_score(estimator, X, dummy_y, cv=kfold)

# calculate predictions
predictions = estimator.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

#trained_classifier = estimator.fit(X, Y)
#print(type(estimator))

#cm = confusion_matrix(Y, y_pred)
#print(cm)
#print("total: " + str(cm.sum()))
#print("accuracy: " + str(numpy.trace(cm) / cm.sum()))
#print("Matthews correlation coefficient: " + str(matthews_corrcoef(Y, y_pred)))



#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/100
 - 0s - loss: 1.9688 - acc: 0.8065
Epoch 2/100
 - 0s - loss: 0.0990 - acc: 0.9858
Epoch 3/100
 - 0s - loss: 0.0441 - acc: 0.9913
Epoch 4/100
 - 0s - loss: 0.0233 - acc: 0.9947
Epoch 5/100
 - 0s - loss: 0.0204 - acc: 0.9953
Epoch 6/100
 - 0s - loss: 0.0186 - acc: 0.9960
Epoch 7/100
 - 0s - loss: 0.0174 - acc: 0.9961
Epoch 8/100
 - 0s - loss: 0.0165 - acc: 0.9961
Epoch 9/100
 - 0s - loss: 0.0151 - acc: 0.9971
Epoch 10/100
 - 0s - loss: 0.0142 - acc: 0.9971
Epoch 11/100
 - 0s - loss: 0.0137 - acc: 0.9969
Epoch 12/100
 - 0s - loss: 0.0131 - acc: 0.9976
Epoch 13/100
 - 0s - loss: 0.0125 - acc: 0.9977
Epoch 14/100
 - 0s - loss: 0.0125 - acc: 0.9975
Epoch 15/100
 - 0s - loss: 0.0120 - acc: 0.9977
Epoch 16/100
 - 0s - loss: 0.0118 - acc: 0.9979
Epoch 17/100
 - 0s - loss: 0.0117 - acc: 0.9979
Epoch 18/100
 - 0s - loss: 0.0115 - acc: 0.9981
Epoch 19/100
 - 0s - loss: 0.0113 - acc: 0.9981
Epoch 20/100
 - 0s - loss: 0.0111 - acc: 0.9981
Epoch 21/100
 - 0s - loss: 0.0113 - acc: 0.9979
E

In [7]:
from pandas_ml import ConfusionMatrix

cm1 = ConfusionMatrix(Y, y_pred)
print(cm1)

NameError: name 'y_pred' is not defined

In [8]:
print(Y)

[0 1 0 ... 1 1 1]


In [9]:
def true_positive_rate(cm, i, total):
    return cm[i][i]/total
    
def false_positive_rate(cm,j):
    fp_rate = 0
    for i in range(0,len(cm)):
        if (i != j):
            fp_rate += cm[i][j]
    if ((fp_rate + cm[j][j]) != 0):
        return fp_rate/(fp_rate + cm[j][j])
    else:
        return 0;
        
def false_negative_rate(cm, i):
    fn_rate = 0;
    for j in range(0,len(cm)):
        if (i != j):
            fn_rate += cm[i][j]
    if ((fn_rate + cm[j][j]) != 0):
        return fn_rate/(fn_rate + cm[j][j])
    else:
        return 0;

def true_negative_rate(cm,i,total):
    tn_rate = 0
    for j in range(0,len(cm)):
        for k in range(0,len(cm)):
            if (j != i and k != i):
                tn_rate += cm[j][k]
    return tn_rate/total

def misclassification_rate(cm,l):
    fp_rate = 0
    fn_rate = 0
    for i in range(0,len(cm)):
        if (i != l):
            fp_rate += cm[i][l]
    for j in range(0,len(cm)):
        if (l != j):
            fn_rate += cm[l][j]
    return (fp_rate + fn_rate)/(fp_rate + fn_rate + cm[l][l])
    
def avg_true_positive_rate(cm):
    tp_rate = 0
    for i in range(0,len(cm)):
        tp_rate += true_positive_rate(cm,i,cm.sum())
    return tp_rate/len(cm)

def avg_false_positive_rate(cm):
    fp_rate = 0
    for i in range(0,len(cm)):
        fp_rate += false_positive_rate(cm,i)
    return fp_rate/len(cm)

def avg_false_negative_rate(cm):
    fn_rate = 0
    for i in range(0,len(cm)):
        fn_rate += false_negative_rate(cm,i)
    return fn_rate/len(cm)

def avg_true_negative_rate(cm):
    tn_rate = 0
    for i in range(0,len(cm)):
        tn_rate += true_negative_rate(cm,i,cm.sum())
    return tn_rate/len(cm)

def avg_misclassification_rate(cm):
    mc_rate = 0
    for i in range(0,len(cm)):
        mc_rate += misclassification_rate(cm,i)
    return mc_rate/len(cm)

def matthews(TP,TN,FP,FN):
    if ((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN) == 0):
        return 0
    return (TP*TN - FP*FN)/math.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))

def print_table(cm):
    print('{:15}'.format('Field'), end='')
    print('{:15}'.format('TP'), end='')
    print('{:15}'.format('FP'), end='')
    print('{:15}'.format('FN'), end='')
    print('{:15}'.format('TN'), end='')
    print('{:15}'.format('MC Rate'), end='')
    print('{:15}'.format('MCC'), end='')
    print()
    print('---------------------------------------------------------------------------------------------------')
    print()
    for i in range(0,len(cm)):
        print('{:15}'.format(attack_labels[i]), end='')
        print('{:15}'.format('{:.5f}'.format(true_positive_rate(cm,i,cm.sum()))), end='')
        print('{:15}'.format('{:.5f}'.format(false_positive_rate(cm,i))), end='')
        print('{:15}'.format('{:.5f}'.format(false_negative_rate(cm,i))), end='')
        print('{:15}'.format('{:.5f}'.format(true_negative_rate(cm,i,cm.sum()))), end='')
        print('{:15}'.format('{:.5f}'.format(misclassification_rate(cm,i))), end='')
        print('{:15}'.format('{:.5f}'.format(matthews(true_positive_rate(cm,i,cm.sum()),true_negative_rate(cm,i,cm.sum()),
                                                      false_positive_rate(cm,i),false_negative_rate(cm,i)))),end='')
        print()
    print()

print_table(cm)
print("Average true positive rate: " + str(avg_true_positive_rate(cm)))
print("Average false positive rate: " + str(avg_false_positive_rate(cm)))
print("Average false negative rate: " + str(avg_false_negative_rate(cm)))
print("Average true negative rate: " + str(avg_true_negative_rate(cm)))
print("Average Misclassification Rate: " + str(avg_misclassification_rate(cm)))
print("Matthews Correlation Coefficient: " + str(matthews(avg_true_positive_rate(cm),avg_true_negative_rate(cm),avg_false_positive_rate(cm),avg_false_negative_rate(cm))))





NameError: name 'cm' is not defined

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(Y, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=attack_labels,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=attack_labels, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

NameError: name 'y_pred' is not defined

In [17]:
sample = np.array([[0.001], [0.014], [0.992], [0.003], [0.011], [0.028], [0.010], [0.006], [0.005], [0.006], [0.004], [0.005], [0.852], [0.995], [0.007], [0.005], [0.003], [0.015], [0.008], [0.027], [0.007], [0.010], [0.994], [0.016], [0.007], [0.003], [0.008], [0.020], [0.131], [0.005], [0.014], [0.006], [0.975], [0.008], [0.884], [0.007], [0.840], [0.006], [0.002], [0.005], [0.009]])
sample = np.resize(sample,(41,))
print(sample.shape)

(41,)


In [18]:
#print(type(X))
#print(X[0],Y[0])



predicted = estimator.predict(sample,batch_size=0)


ValueError: Error when checking input: expected dense_13_input to have shape (41,) but got array with shape (1,)